In [1]:
import os
from typing import Callable

from edgedroid.models.sampling import *
from edgedroid.models.timings import *

rtts = np.linspace(0, 5, 11)[1:]

runs_per_model = 60
task_steps = 100

timing_models: Dict[str, Callable[[], ExecutionTimeModel]] = {
    # "empirical-low": lambda: EmpiricalExecutionTimeModel.from_default_data(neuroticism=0.0),
    "empirical-high": lambda: EmpiricalExecutionTimeModel.from_default_data(neuroticism=1.0),
    # "theoretical-low": lambda: TheoreticalExecutionTimeModel.from_default_data(neuroticism=0.0),
    # "theoretical-high": lambda: TheoreticalExecutionTimeModel.from_default_data(neuroticism=1.0),
    # "constant": lambda: ConstantExecutionTimeModel.from_default_data(),
    # "naive": lambda: NaiveExecutionTimeModel.from_default_data(),
    # "fitted-naive": lambda: FittedNaiveExecutionTimeModel.from_default_data(),
}

In [2]:
import edgedroid.data as e_data
data, *_ = e_data.load_default_exec_time_data()
data

,run_id,ttf,exec_time,neuroticism
0,134146,0.597441,3.654797,0.375
1,134146,0.553513,4.438645,0.375
2,134146,0.561716,2.943222,0.375
3,134146,0.586512,5.405761,0.375
4,134146,0.558940,5.225161,0.375
...,...,...,...,...
6755,137353,0.557074,6.439071,0.625
6756,137353,0.534339,4.680858,0.625
6757,137353,0.560288,3.467878,0.625
6758,137353,0.579000,2.325759,0.625


In [3]:
exec_time_qs = np.round(data["exec_time"].describe(percentiles=[0.25, 0.5, 0.75])[["25%", "50%", "75%"]], decimals=1)
exec_time_qs

25%    3.9
50%    5.2
75%    7.0
Name: exec_time, dtype: float64

In [4]:
from edgedroid.models.sampling.adaptive import _aperiodic_instant_iterator
from typing import NamedTuple
import scipy.stats as stats

class SamplingResult(NamedTuple):
    duration: float
    num_samples: int

class ExpSampling(BaseFrameSamplingModel, abc.ABC):
    @abc.abstractmethod
    def constant_rtt_sampling(self, rtt: float, prev_ttf: float, target_exec_time: float) -> SamplingResult:
        pass


class GreedySampling(ZeroWaitFrameSamplingModel, ExpSampling):
    def constant_rtt_sampling(self, rtt: float, prev_ttf: float, target_exec_time: float) -> SamplingResult:
        num_samples = 1
        instant = 0.0

        while instant <= target_exec_time:
            instant += rtt
            num_samples += 1

        return SamplingResult(instant + rtt, num_samples)

class IdealSampling(IdealFrameSamplingModel, ExpSampling):
    def constant_rtt_sampling(self, rtt: float, prev_ttf: float, target_exec_time: float) -> SamplingResult:
        return SamplingResult(target_exec_time + rtt, 1)

class PeriodicSampling(RegularFrameSamplingModel, ExpSampling):
    def constant_rtt_sampling(self, rtt: float, prev_ttf: float, target_exec_time: float) -> SamplingResult:
        num_samples = 1
        instant = self._interval

        interval = max(self._interval, rtt)

        while instant <= target_exec_time:
            instant += interval
            num_samples += 1

        return SamplingResult(instant + rtt, num_samples)

class HoldSampling(HoldFrameSamplingModel, ExpSampling):
    def constant_rtt_sampling(self, rtt: float, prev_ttf: float, target_exec_time: float) -> SamplingResult:
        num_samples = 1
        instant = self._hold_time

        while instant <= target_exec_time:
            instant += rtt
            num_samples += 1

        return SamplingResult(instant + rtt, num_samples)

class AdaptiveSamplingMixin(BaseAperiodicFrameSamplingModel, ExpSampling, abc.ABC):
    def constant_rtt_sampling(self, rtt: float, prev_ttf: float, target_exec_time: float) -> SamplingResult:
        self._timing_model.advance(prev_ttf)
        alpha = self.get_alpha()
        beta = self.get_beta()

        instant_iter = _aperiodic_instant_iterator(
            mu=self._timing_model.get_expected_execution_time(),
            alpha=alpha,
            beta=beta,
        )
        # first instant doesn't depend on RTT
        instant = next(instant_iter)
        num_samples = 1

        while instant <= target_exec_time:
            instant = max(next(instant_iter), instant + rtt)
            num_samples += 1

        self.record_rtts([rtt] * num_samples)
        return SamplingResult(instant + rtt, num_samples)

class AdaptiveSampling(AperiodicFrameSamplingModel, AdaptiveSamplingMixin):
    pass

class AdaptivePowerSampling(AperiodicPowerFrameSamplingModel, AdaptiveSamplingMixin):
    pass

sampling_schemes: Dict[str, Callable[[], ExpSampling]] = {
    "greedy": lambda : GreedySampling.from_default_data(),
    "ideal": lambda : IdealSampling.from_default_data(),
}

sampling_schemes.update({
    "adaptive-empirical": lambda : AdaptiveSampling.from_default_data(EmpiricalExecutionTimeModel.from_default_data(neuroticism=None)),
    # "adaptive-empirical-low": lambda : AdaptiveSampling.from_default_data(EmpiricalExecutionTimeModel.from_default_data(neuroticism=0.0)),
    # "adaptive-empirical-high": lambda : AdaptiveSampling.from_default_data(EmpiricalExecutionTimeModel.from_default_data(neuroticism=1.0)),
    "adaptive-theoretical-exgaussian": lambda : AdaptiveSampling.from_default_data(
        TheoreticalExecutionTimeModel.from_default_data(neuroticism=None, distribution=stats.exponnorm)
    ),
    "adaptive-theoretical-rayleigh": lambda : AdaptiveSampling.from_default_data(
        TheoreticalExecutionTimeModel.from_default_data(neuroticism=None, distribution=stats.rayleigh)
    ),
    # "adaptive-theoretical-low": lambda : AdaptiveSampling.from_default_data(TheoreticalExecutionTimeModel.from_default_data(neuroticism=0.0)),
    # "adaptive-theoretical-high": lambda : AdaptiveSampling.from_default_data(TheoreticalExecutionTimeModel.from_default_data(neuroticism=1.0)),
    "adaptive-fitted-naive-exgaussian": lambda : AdaptiveSampling.from_default_data(FittedNaiveExecutionTimeModel.from_default_data(dist=stats.exponnorm)),
    # "adaptive-fitted-naive-rayleigh": lambda : AdaptiveSampling.from_default_data(FittedNaiveExecutionTimeModel.from_default_data(dist=stats.rayleigh))
})

sampling_schemes.update({
    f"adaptive-constant-Q{i + 1}-{t:0.1f}s":
        lambda : AdaptiveSampling.from_default_data(ConstantExecutionTimeModel(float(t))) for i, t in enumerate(exec_time_qs)
})

sampling_schemes.update({
    "adaptive-power-empirical": lambda : AdaptivePowerSampling.from_default_data(EmpiricalExecutionTimeModel.from_default_data(neuroticism=None)),
    # "adaptive-power-empirical-low": lambda : AdaptivePowerSampling.from_default_data(EmpiricalExecutionTimeModel.from_default_data(neuroticism=0.0)),
    # "adaptive-power-empirical-high": lambda : AdaptivePowerSampling.from_default_data(EmpiricalExecutionTimeModel.from_default_data(neuroticism=1.0)),
     "adaptive-power-theoretical-exgaussian": lambda : AdaptivePowerSampling.from_default_data(
        TheoreticalExecutionTimeModel.from_default_data(neuroticism=None, distribution=stats.exponnorm)
    ),
    "adaptive-power-theoretical-rayleigh": lambda : AdaptivePowerSampling.from_default_data(
        TheoreticalExecutionTimeModel.from_default_data(neuroticism=None, distribution=stats.rayleigh)
    ),
    # "adaptive-power-theoretical-low": lambda : AdaptivePowerSampling.from_default_data(TheoreticalExecutionTimeModel.from_default_data(neuroticism=0.0)),
    # "adaptive-power-theoretical-high": lambda : AdaptivePowerSampling.from_default_data(TheoreticalExecutionTimeModel.from_default_data(neuroticism=1.0)),
    "adaptive-power-fitted-naive-exgaussian": lambda : AdaptivePowerSampling.from_default_data(FittedNaiveExecutionTimeModel.from_default_data(dist=stats.exponnorm)),
    # "adaptive-power-fitted-naive-rayleigh": lambda : AdaptivePowerSampling.from_default_data(FittedNaiveExecutionTimeModel.from_default_data(dist=stats.rayleigh))
})

sampling_schemes.update({
    f"adaptive-power-constant-Q{i + 1}-{t:0.1f}s":
        lambda : AdaptivePowerSampling.from_default_data(ConstantExecutionTimeModel(float(t))) for i, t in enumerate(exec_time_qs)
})

sampling_schemes.update({
    f"periodic-{t:0.1f}s": lambda : PeriodicSampling.from_default_data(sampling_interval_seconds=float(t)) for t in (1, 3, 5)
})
sampling_schemes.update({
    f"hold-{t:0.1f}s": lambda : HoldSampling.from_default_data(hold_time_seconds=float(t)) for t in (1, 3, 5)
})

list(sampling_schemes.keys())

['greedy',
 'ideal',
 'adaptive-empirical',
 'adaptive-theoretical-exgaussian',
 'adaptive-theoretical-rayleigh',
 'adaptive-fitted-naive-exgaussian',
 'adaptive-constant-Q1-3.9s',
 'adaptive-constant-Q2-5.2s',
 'adaptive-constant-Q3-7.0s',
 'adaptive-power-empirical',
 'adaptive-power-theoretical-exgaussian',
 'adaptive-power-theoretical-rayleigh',
 'adaptive-power-fitted-naive-exgaussian',
 'adaptive-power-constant-Q1-3.9s',
 'adaptive-power-constant-Q2-5.2s',
 'adaptive-power-constant-Q3-7.0s',
 'periodic-1.0s',
 'periodic-3.0s',
 'periodic-5.0s',
 'hold-1.0s',
 'hold-3.0s',
 'hold-5.0s']

In [5]:
power_mw = {
    "comm": 0.045,
    "idle": 0.015
}  # Watts

proc_time = 0.3 # 300 ms
warmup_steps = 20

def run_combination(timing: str, sampling: str, rtt: float, repetition: int) -> pd.DataFrame:
    timing_model = timing_models[timing]()
    sampling_model = sampling_schemes[sampling]()
    prev_ttf = rtt
    cumulative_duration = 0.0
    cumulative_samples = 0

    # for power calculations:
    comm_time_per_sample = rtt - proc_time
    cumulative_energy = 0.0

    rows = deque()

    for step in range(1, task_steps + warmup_steps + 1):
        exec_time = timing_model.advance(prev_ttf).get_execution_time()
        duration, num_samples = sampling_model.constant_rtt_sampling(rtt, prev_ttf, exec_time)

        if step < warmup_steps:
            continue

        # duration = final_sample + rtt
        cumulative_duration += duration
        cumulative_samples += num_samples
        ttf = duration - exec_time
        wait_time = ttf - rtt

        # calculate power
        total_comm_time = comm_time_per_sample * num_samples
        total_idle_time = duration - total_comm_time

        total_energy = (total_idle_time * power_mw["idle"]) + (total_comm_time * power_mw["comm"])
        cumulative_energy += total_energy

        rows.append(
            {
                "timing_model": timing,
                "sampling_scheme": sampling,
                "rtt": rtt,
                "step": step,
                "previous_ttf": prev_ttf,
                "execution_time": exec_time,
                "step_duration": duration,
                "ttf": ttf,
                "wait_time": wait_time,
                "samples": num_samples,
                "cumulative_duration": cumulative_duration,
                "cumulative_samples": cumulative_samples,
                "repetition": repetition,
                "energy": total_energy,
                "cumulative_energy": cumulative_energy,
            }
        )
        prev_ttf = ttf

    return pd.DataFrame(rows)

In [6]:
from typing import Tuple
from tqdm.notebook import tqdm
import multiprocess as mp
import shutil

result_path = "./sampling_scaling_rtt.gzip"
combs = set(itertools.product(timing_models.keys(), sampling_schemes.keys(), rtts, range(1, runs_per_model + 1)))

# only calculate missing results
results = deque()
try:
    old_results = pd.read_parquet(result_path)
    existing_combinations = set(old_results[["timing_model", "sampling_scheme", "rtt", "repetition"]].itertuples(index=False))
    shutil.rmtree(result_path)
    results.append(old_results)
except FileNotFoundError:
    existing_combinations = set()

combs.difference_update(existing_combinations)
if len(combs) == 0:
    print("No missing combinations.")
else:
    # noinspection PyUnresolvedReferences
    with tqdm(total=len(combs), bar_format="{l_bar}{bar}{n_fmt}/{total_fmt} [Time: {elapsed}]") as bar, mp.Pool(os.cpu_count() - 2) as pool:
        bar.set_description("Running timing model/sampling scheme combinations...")

        def make_callback(cmb: Tuple[str, str, float]) -> Callable[[...], None]:
            def _cb(*args, **kwargs) -> None:
                bar.update()
                bar.set_description(f"Running timing model/sampling scheme combinations...")

            return _cb

        procs = [
            pool.apply_async(run_combination, args=c, callback=make_callback(c)) for c in combs
        ]

        for p in procs:
            results.append(p.get())

results = pd.concat(results, ignore_index=True)

results["timing_model"] = results["timing_model"].astype(
    pd.CategoricalDtype(["fitted-naive", "empirical-low", "empirical-high", "theoretical-low", "theoretical-high"], ordered=True)
)
results["sampling_scheme"] = results["sampling_scheme"].astype(
    pd.CategoricalDtype(sampling_schemes.keys(), ordered=False)
)
results.to_parquet(result_path, partition_cols=["timing_model", "sampling_scheme"], compression="gzip")
results

No missing combinations.


,rtt,step,previous_ttf,execution_time,step_duration,ttf,wait_time,samples,cumulative_duration,cumulative_samples,repetition,energy,cumulative_energy,timing_model,sampling_scheme
0,2.5,1,2.500000,4.201379,8.910716,4.709337,2.209337,3,8.910716,3,4,0.331661,0.331661,empirical-high,adaptive-constant-Q1-3.9s
1,2.5,2,4.709337,15.396027,18.753317,3.357290,0.857290,6,27.664033,9,4,0.677300,1.008961,empirical-high,adaptive-constant-Q1-3.9s
2,2.5,3,3.357290,6.658278,11.242213,4.583935,2.083935,3,38.906246,12,4,0.366633,1.375594,empirical-high,adaptive-constant-Q1-3.9s
3,2.5,4,4.583935,19.401157,24.021556,4.620399,2.120399,8,62.927802,20,4,0.888323,2.263917,empirical-high,adaptive-constant-Q1-3.9s
4,2.5,5,4.620399,2.608902,6.409484,3.800582,1.300582,1,69.337286,21,4,0.162142,2.426059,empirical-high,adaptive-constant-Q1-3.9s
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647995,0.5,41,2.232731,5.934689,10.500000,4.565311,4.065311,2,340.500000,64,47,0.169500,5.491500,empirical-high,periodic-5.0s
647996,0.5,42,4.565311,6.944365,10.500000,3.555635,3.055635,2,351.000000,66,47,0.169500,5.661000,empirical-high,periodic-5.0s
647997,0.5,43,3.555635,6.109906,10.500000,4.390094,3.890094,2,361.500000,68,47,0.169500,5.830500,empirical-high,periodic-5.0s
647998,0.5,44,4.390094,6.356987,10.500000,4.143013,3.643013,2,372.000000,70,47,0.169500,6.000000,empirical-high,periodic-5.0s
